## Title 

Description

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
from random import random

from SourceCodeTools.models.training_config import get_config, save_config, load_config
from SourceCodeTools.code.data.dataset.Dataset import SourceGraphDataset, filter_dst_by_freq
from SourceCodeTools.models.graph.train.sampling_multitask2 import training_procedure, SamplingMultitaskTrainer
from SourceCodeTools.models.graph.train.objectives.NodeClassificationObjective import NodeClassifierObjective
from SourceCodeTools.models.graph.train.objectives.SubgraphClassifierObjective import SubgraphAbstractObjective, \
    SubgraphClassifierObjective, SubgraphEmbeddingObjective
from SourceCodeTools.models.graph.train.utils import get_name, get_model_base
from SourceCodeTools.models.graph import RGGAN
from SourceCodeTools.tabular.common import compact_property
from SourceCodeTools.code.data.file_utils import unpersist

import dgl
import torch
import numpy as np
from argparse import Namespace
from torch import nn
from datetime import datetime
from os.path import join
from functools import partial

Using backend: pytorch


# Prepare parameters and options

Full list of options that can be added can be found in `SourceCodeTools/models/training_options.py`. They are ment to be used as arguments for cli trainer. Trainer script can be found in `SourceCodeTools/scripts/train.py`.

For the task of subgraph classification the important options are:
- `subgraph_partition` is path to subgraph-based train/val/test sets. Storead as Dataframe with subgraph id and partition mask
- `subgraph_id_column` is a column is `common_edges` file that stores subgraph id.
- For variable misuse task (same will apply to authorship attribution) subgraphs are created for individual functions (files for SCAA). The label is stored in `common_filecontent`.

In [3]:
tokenizer_path = "sentencepiece_bpe.model"

data_path = "10_percent_v1"
subgraph_partition = join(data_path, "partition.json.bz2")
filecontent_path = join(data_path, "common_filecontent.json.bz2")

# Data distribution

In [4]:
labels = unpersist(filecontent_path)
for part in labels['partition'].unique():
    print(part)
    print(labels.query('partition == @part')['label'].value_counts())
    print()

train
Variable misuse    35036
Correct            35036
Name: label, dtype: int64

dev
Correct            3754
Variable misuse    3754
Name: label, dtype: int64

eval
Variable misuse    18787
Correct            18787
Name: label, dtype: int64



# Config

In [5]:
config = get_config(
    subgraph_id_column="file_id",
    data_path=data_path,
    model_output_dir=data_path,
    subgraph_partition=subgraph_partition,
    tokenizer_path=tokenizer_path,
    objectives="subgraph_clf",
    #use_edge_types=True,
    gpu=-1,
    epochs=2, 
    
    train_frac=0.8,
    random_seed=42, 
    
    # model parameters
    elem_emb_size=300,
    node_emb_size=300,                  # *** dimensionality of node embeddings
    h_dim=300,                           # *** should match to node dimensionality
    n_layers=5,
    dropout=0.1,
    activation="tanh"
)

In [6]:
save_config(config, "var_misuse_subgraph.yaml")

In [7]:
config

{'DATASET': {'data_path': '10_percent_v1',
  'train_frac': 0.8,
  'filter_edges': None,
  'min_count_for_objectives': 5,
  'self_loops': False,
  'use_node_types': False,
  'use_edge_types': False,
  'no_global_edges': False,
  'remove_reverse': False,
  'custom_reverse': None,
  'restricted_id_pool': None,
  'random_seed': 42,
  'subgraph_id_column': 'file_id',
  'subgraph_partition': '10_percent_v1\\partition.json.bz2'},
 'TRAINING': {'model_output_dir': '10_percent_v1',
  'pretrained': None,
  'pretraining_phase': 0,
  'sampling_neighbourhood_size': 10,
  'neg_sampling_factor': 3,
  'use_layer_scheduling': False,
  'schedule_layers_every': 10,
  'elem_emb_size': 300,
  'embedding_table_size': 200000,
  'epochs': 2,
  'batch_size': 128,
  'learning_rate': 0.001,
  'objectives': 'subgraph_clf',
  'save_each_epoch': False,
  'save_checkpoints': True,
  'early_stopping': False,
  'early_stopping_tolerance': 20,
  'force_w2v_ns': False,
  'use_ns_groups': False,
  'nn_index': 'brute',
  

# Create Dataset

In [8]:
dataset = SourceGraphDataset(
    **{**config["DATASET"], **config["TOKENIZER"]}
)

# Declare target loading function (labels)

In [9]:
def load_labels():
    filecontent = unpersist(filecontent_path)
    return filecontent[["id", "label"]].rename({"id": "src", "label": "dst"}, axis=1)

One or several objectives could be used

In [10]:
class Trainer(SamplingMultitaskTrainer):
    def create_objectives(self, dataset, tokenizer_path):
        self.objectives = nn.ModuleList()
        
        self.objectives.append(
            self._create_subgraph_objective(
                objective_name="VariableMisuseClf",
                objective_class=SubgraphClassifierObjective,
                dataset=dataset,
                tokenizer_path=tokenizer_path,
                labels_fn=load_labels,
            )
        )

## Launch Tensorboard

In [11]:
# %tensorboard --logdir './10_percent_v1'

## Start model training and Evaluation

In [12]:
training_procedure(
    dataset, 
    model_name=RGGAN, 
    model_params=config["MODEL"],
    trainer_params=config["TRAINING"],
    model_base_path=get_model_base(config["TRAINING"], get_name(RGGAN, str(datetime.now()))),
    trainer=Trainer
)

KeyboardInterrupt: 